In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL/Challenge2

### Import Libraries

In [ ]:
!pip install tsaug
import tsaug
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
plt.rc('font', size=16)
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### (Unzip the dataset)

In [ ]:
!unzip training_dataset_homework2.zip

### Seed

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Loading the data

In [ ]:
x = np.load('x_train.npy')
y = np.load('y_train.npy')
print(x.shape)
print(y.shape)
print(y)

### Splitting the data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=seed,shuffle=True, test_size=0.15, stratify=y)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

### Preprocessing

In [ ]:
scaler=RobustScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_val = scaler.transform(x_val.reshape(-1, x_val.shape[-1])).reshape(x_val.shape)


### Open the trained scaler

In [ ]:
import pickle
filehandler = open('scaler', 'rb')
scaler = pickle.load(filehandler)
x_train = scaler.transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_val = scaler.transform(x_val.reshape(-1, x_val.shape[-1])).reshape(x_val.shape)

### Save the scaler after you train it the first time

In [ ]:
import pickle
filehandler=open('scaler', 'wb')
pickle.dump(scaler, filehandler)

### Augmentation

In [ ]:
x_train_noisy = tsaug.AddNoise(scale=0.1).augment(X=x_train)
x_final = np.concatenate([x_train, x_train_noisy])
print(x_final.shape)
augmented_labels = np.tile(y_train, (2,))
print(augmented_labels.shape)

In [ ]:
# Convert the sparse labels to categorical values
y_val = tfk.utils.to_categorical(y_val)
augmented_labels = tfk.utils.to_categorical(augmented_labels)
x_train.shape, y_train.shape

In [ ]:
input_shape = x_final.shape[1:]
classes = augmented_labels.shape[-1]
batch_size = 64
epochs = 200

### LSTM Vanilla

In [ ]:
def build_LSTM_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    lstm = tfkl.LSTM(128, return_sequences=True)(input_layer)
    lstm = tfkl.LSTM(128)(lstm)
    dropout = tfkl.Dropout(.5, seed=seed)(lstm)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

### Bilinear LSTM

In [ ]:
def build_BiLSTM_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    bilstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(input_layer)
    bilstm = tfkl.Bidirectional(tfkl.LSTM(128))(bilstm)
    dropout = tfkl.Dropout(.5, seed=seed)(bilstm)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

### 1D Convolutional

In [ ]:
def build_1DCNN_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(128,6,padding='same',activation='relu')(input_layer)
    cnn = tfkl.MaxPooling1D()(cnn)
    cnn = tfkl.Conv1D(128,6,padding='same',activation='relu')(cnn)
    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.5, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

### Attention layer

In [ ]:
import keras.backend as K
from keras.layers import Layer
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,input_layer):
        # Alignment scores. Pass them through tanh function
        e = K.relu(K.dot(input_layer,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = input_layer * alpha
        context = K.sum(context, axis=1)
        return context

### 1D Convolutional Alternative (Best)




In [ ]:
def build_1DCNN_classifierBest(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor 
    cnn = tfkl.Conv1D(100,6,padding='same',activation='relu')(input_layer)
    cnn = tfkl.Conv1D(100,6,padding='same',activation='relu')(cnn)
    cnn = tfkl.Conv1D(100,6,padding='same',activation='relu')(cnn)
    pool = tfkl.MaxPooling1D(pool_size=3)(cnn)
    cnn2 = tfkl.Conv1D(100,6,padding='same',activation='relu')(pool)
    cnn2 = tfkl.Conv1D(100,6,padding='same',activation='relu')(cnn2)
    dropout = tfkl.Dropout(.5, seed=seed)(cnn2)
    attention_layer = attention()(dropout)

    # Classifier
    output_layer = tfkl.Dense(classes, activation='softmax')(attention_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

### ResNet1D

In [ ]:
def build_ResNetmodel( input_shape, nb_classes):
        n_feature_maps = 32

        input_layer = tfk.layers.Input(input_shape)

        # BLOCK 1

        conv_x = tfk.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = tfk.layers.BatchNormalization()(conv_x)
        conv_x = tfk.layers.Activation('relu')(conv_x)

        conv_y = tfk.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = tfk.layers.BatchNormalization()(conv_y)
        conv_y = tfk.layers.Activation('relu')(conv_y)

        conv_z = tfk.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = tfk.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = tfk.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = tfk.layers.BatchNormalization()(shortcut_y)

        output_block_1 = tfk.layers.add([shortcut_y, conv_z])
        output_block_1 = tfk.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = tfk.layers.BatchNormalization()(conv_x)
        conv_x = tfk.layers.Activation('relu')(conv_x)

        conv_y = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = tfk.layers.BatchNormalization()(conv_y)
        conv_y = tfk.layers.Activation('relu')(conv_y)

        conv_z = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = tfk.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = tfk.layers.BatchNormalization()(shortcut_y)

        output_block_2 = tfk.layers.add([shortcut_y, conv_z])
        output_block_2 = tfk.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = tfk.layers.BatchNormalization()(conv_x)
        conv_x = tfk.layers.Activation('relu')(conv_x)

        conv_y = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = tfk.layers.BatchNormalization()(conv_y)
        conv_y = tfk.layers.Activation('relu')(conv_y)

        conv_z = tfk.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = tfk.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = tfk.layers.BatchNormalization()(output_block_2)

        output_block_3 = tfk.layers.add([shortcut_y, conv_z])
        output_block_3 = tfk.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = tfk.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = tfk.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = tfk.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.Adam(),
                      metrics=['accuracy'])

        return model

### Build the model

In [ ]:
model = build_1DCNN_classifierBest(input_shape, classes)
model.summary()

### Train the model

In [ ]:
# Train the model
history = model.fit(
    x = x_final,
    y = augmented_labels,
    validation_data=(x_val, y_val),
    batch_size = batch_size,
    epochs = epochs,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
model.save("1DConvBestAugAttention")

### Plot the training

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()